## 딥러닝 TEST

1. 퍼셉트론은 뉴런을 공학적인 구조로 나타낸것이다.

2. 퍼셉트론 기본동작  
각피쳐에 가중치를 곱고 절편을 더해주어 하나의 퍼셉트론을 구성
P= w1x1+w2x2+w3x3+w4x4 + b   

3. 
이전 층의 결과값을 변환하여 다른 층의 뉴런으로 전달하기 위해서 필요하다.
선형모델을 계속 층을 쌓는다하여도 많은 선형이 합쳐지면 결국 하나의 선형에 가깝게 되기때문에
비선형함수인 활성화함수를 통하여 층을 만들어 주는 역활을 한다.

4.
- Relu : 0보다 작을때 0 나머지는 그대로출력
- leackyRelu : 0보다 작을때 0에 가까운 음수로 만들어줌
- sigmoid : 0 ~ 1사이 값
- step : 0과 1로 나타냄


5. 함수의 기울기를 구한다음 기울기의 반대 방향으로 이동하여 기울기가 최소가 되는 지점을 찾는방법이다.
- Batch 경사하강법 : 학습데이터를 배치크기로 나눠서 학습시키는 방식
- 확률 경사하강법 : 학습 데이터에서 무작위로 샘플데이터를 추출하여 학습하는 방식
- adagrad : 학습률을 자동으로 조정하는 방식

6. 회귀 모델 구현

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class regression(nn.module):
    def __init__(self):
        super().__init__()

        self.input_layer=nn.Linear(3,5)   # 입력층  입력 3 (피쳐3개)         출력 5
        self.h_layer = nn.Linear(5,10)    # 은닉층  입력  5                 출력 10
        self.out_layer = nn.Linear(10,1)  # 출력층  입력 10                  출력 1     # 회귀니까 출력1

    # 순방향 학습
    def forward(self,x):
        y = F.relu(self.input_layer(x))     
        y = F.relu(self.h_layer(y))
        return self.out_layer(y)     


7. 이진 분류 모델 구현

In [ ]:
# 모델을 만들때 은닉의 퍼셉트론을 리스트로 준다고 가정 
class BNClassfication(nn.module):

    def __init__(self,h_list=[]):
        super().__init__()

        self.input_layer = nn.Linear(5,h_list[0])
        self.h_layers =nn.ModuleList()

        for i in range(3):      
            self.h_layers.append(nn.Linear(h_list[i],h_list[i+1]))
        
        self.out_layers = nn.Linear(h_list[-1],1)
    
    def forward(self,x):
        y = F.relu(self.input_layer(x))  

        for h_layer in self.h_layers:
            y=F.relu(h_layer(y))   
        
        return F.sigmoid(self.out_layers(y))  # 이진분류는 sigmoid가 좋다다


8. 다중 분류 모델 구현

In [ ]:
# 모델을 만들때 은닉의 퍼셉트론을 리스트로 준다고 가정 
class MClassfication(nn.module):
    
    def __init__(self,h_list=[]):
        super().__init__()

        self.input_layer = nn.Linear(5,h_list[0])  # 피쳐 5개
        self.h_layers = nn.ModuleList()

        for i in range(len(h_list)-1):
            self.h_layers.append(nn.Linear(h_list[i],h_list[i+1]))

        self.out_layers = nn.Linear(h_list[-1],8)     # 클래스 8개
    
    def forward(self,x):
        y = F.relu(self.input_layer(x))  

        for h_layer in self.h_layers:
            y=F.relu(h_layer(y))   
        
        return F.softmax(self.out_layers(y), dim=1)     # 다중분류라서 softmax

9. 기울기 소실 개념 및 해결 방법
- 역전파 과정에서 기울기값이 매우 작아져 다음층으로 전달 될때 기울기가 거의 소실 되어 입력층 가까운층에서는 학습이 잘 되지 않는것을 말한다.
- 해결방법 : 적절한 활성화함수를 사용하거나 가중치 초기화 등을 사용하여 기울기 소실을 해결 할 수 있다

10.
- model.train() 학습모드
    * 학습과정에서 사용되는 정규화 기법들이 동작하도록 한다.
    * 역전파를 진행하고 기울기를 계산한다.
- model.eval() 검증모드
    * 검증과정에서 사용하는 것으로 정규화 기법(드랍아웃)이 동작하지 않는다.